# ai3 for the same thing

## dependencies

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## dataset stuff

### read the dataset

In [2]:
df = pd.read_csv('dataset.csv')
df = df.sample(frac = 1)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
216,46,0,2,105,204,0,0,172,0,0.0,1,0.0,3.0,0
90,66,1,4,120,302,0,2,151,0,0.4,2,0.0,3.0,0
159,68,1,3,118,277,0,0,151,0,1.0,1,1.0,7.0,0
233,74,0,2,120,269,0,2,121,1,0.2,1,1.0,3.0,0
114,62,0,3,130,263,0,0,97,0,1.2,2,1.0,7.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,54,1,3,125,273,0,2,152,0,0.5,3,1.0,3.0,0
267,59,1,3,126,218,1,0,134,0,2.2,2,1.0,6.0,2
153,55,1,4,160,289,0,2,145,1,0.8,2,1.0,7.0,4
19,49,1,2,130,266,0,0,171,0,0.6,1,0.0,3.0,0


### train_test split

In [3]:
ratio = 0.9 # 90:10 train:test ratio
train_set = df.sample(frac=ratio)

# Dropping all those indexes from the dataframe that exists in the train_set
test_set = df.drop(train_set.index)
train_set.shape, test_set.shape

train_set
test_set

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,diagnosis
40,65,0,4,150,225,0,2,114,0,1.0,2,3.0,7.0,4
156,51,1,4,140,299,0,0,173,1,1.6,1,0.0,7.0,1
98,52,1,2,134,201,0,0,158,0,0.8,1,1.0,3.0,0
243,61,1,1,134,234,0,0,145,0,2.6,2,2.0,3.0,2
240,41,1,2,110,235,0,0,153,0,0.0,1,0.0,3.0,0
212,41,1,3,130,214,0,2,168,0,2.0,2,0.0,3.0,0
251,58,1,4,146,218,0,0,105,0,2.0,2,1.0,7.0,1
190,50,1,3,129,196,0,0,163,0,0.0,1,0.0,3.0,0
234,54,0,3,160,201,0,0,163,0,0.0,1,1.0,3.0,0
280,57,1,4,110,335,0,0,143,1,3.0,2,1.0,7.0,2


### formating

In [6]:
# import training dataset

# Clean column names
train_set.columns = train_set.columns.str.strip()

# Handle missing values represented by '?'
train_set = train_set.replace('?', np.nan)

# Convert columns to numeric, drop rows with NaNs
train_set = train_set.dropna()
train_set = train_set.astype(float)

# Split features and labels
X = train_set.drop("diagnosis", axis=1)
y = train_set["diagnosis"]

# Convert to PyTorch tensors
inputs = torch.tensor(X.values, dtype=torch.float32)
targets = torch.tensor(y.values, dtype=torch.float32)  # or LongTensor if it's class labels

print(inputs.shape, targets.shape)

torch.Size([267, 13]) torch.Size([267])


In [7]:
# import testing dataset

# Clean column names
test_set.columns = test_set.columns.str.strip()

# Handle missing values represented by '?'
test_set = test_set.replace('?', np.nan)

# Convert columns to numeric, drop rows with NaNs
test_set = test_set.dropna()
test_set = test_set.astype(float)

# Split features and labels
X = test_set.drop("diagnosis", axis=1)
y = test_set["diagnosis"]

# Convert to PyTorch tensors
test_tensor = torch.tensor(X.values, dtype=torch.float32)
test_target = torch.tensor(y.values, dtype=torch.float32)  # or LongTensor if it's class labels

print(test_tensor.shape, test_target.shape)

torch.Size([30, 13]) torch.Size([30])


## ai

### parameters for layers (basically setup)

In [8]:
class SimpleNN(nn.Module): 
    def __init__(self): 
        super(SimpleNN, self).__init__() 
        self.fc1 = nn.Linear(13, 8)  # input layer → hidden layer
        self.relu = nn.ReLU()       # activation
        self.fc2 = nn.Linear(8, 1)  # hidden layer → output layer

    def forward(self, x): 
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)  # <-- you forgot to call fc3 with (x)
        return x

# Instantiate model
model = SimpleNN()
print(model)

SimpleNN(
  (fc1): Linear(in_features=13, out_features=8, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)


### training

In [9]:
#calculate mean squared error
criterion = nn.MSELoss()
optimiser = optim.SGD(model.parameters(), lr=0.01)

In [10]:
# acc. training
for epoch in range(1000):
    optimiser.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    if torch.isnan(loss):
        print("Loss is NaN!")
        break
    loss.backward()
    optimiser.step()
    
    print(f'Epoch [{epoch + 1}/1000], Loss: {loss.item():.4f}')
print(outputs)

/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([267])) that is different to the input size (torch.Size([267, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/1000], Loss: 8.1487
Epoch [2/1000], Loss: 1714107.1250
Epoch [3/1000], Loss: 3283630.2500
Epoch [4/1000], Loss: 127.4135
Epoch [5/1000], Loss: 122.4281
Epoch [6/1000], Loss: 117.6401
Epoch [7/1000], Loss: 113.0418
Epoch [8/1000], Loss: 108.6255
Epoch [9/1000], Loss: 104.3842
Epoch [10/1000], Loss: 100.3108
Epoch [11/1000], Loss: 96.3987
Epoch [12/1000], Loss: 92.6415
Epoch [13/1000], Loss: 89.0331
Epoch [14/1000], Loss: 85.5676
Epoch [15/1000], Loss: 82.2393
Epoch [16/1000], Loss: 79.0428
Epoch [17/1000], Loss: 75.9729
Epoch [18/1000], Loss: 73.0246
Epoch [19/1000], Loss: 70.1930
Epoch [20/1000], Loss: 67.4736
Epoch [21/1000], Loss: 64.8619
Epoch [22/1000], Loss: 62.3535
Epoch [23/1000], Loss: 59.9445
Epoch [24/1000], Loss: 57.6309
Epoch [25/1000], Loss: 55.4089
Epoch [26/1000], Loss: 53.2750
Epoch [27/1000], Loss: 51.2255
Epoch [28/1000], Loss: 49.2572
Epoch [29/1000], Loss: 47.3668
Epoch [30/1000], Loss: 45.5513
Epoch [31/1000], Loss: 43.8076
Epoch [32/1000], Loss: 42.1330
E

### testing

## ui